<a href="https://colab.research.google.com/github/zfe-dev/PySpark-GoogleColab-Introduction/blob/main/PySpark_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1: Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%ls

Mounted at /content/drive
drive/  sample_data/


##1.1: If not in the good folder, moves to it

In [11]:
%cd /content/drive/MyDrive/Github/PySpark_Introduction/


/content/drive/MyDrive/Github/PySpark_Introduction


#2: Import and setup PySpark

In [12]:
!pip install pyspark findspark

In [4]:
import findspark
findspark.init()

#3: Name the project and let's go!

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark-Introduction").getOrCreate()

#4: Read the CSV file available in the driver folder

In [9]:
%ls
df = spark.read.csv("new_retail_data.csv")

new_retail_data.csv  PySpark_Introduction.ipynb


In [10]:
df.show(10)

+--------------+-----------+-------------------+-------------------+------------+--------------------+----------+---------------+-------+---------+----+------+------+----------------+----------+------+---------+--------+---------------+-----------+------------+----------------+-------------+------------+---------+---------------+--------------+------------+-------+-----------------+
|           _c0|        _c1|                _c2|                _c3|         _c4|                 _c5|       _c6|            _c7|    _c8|      _c9|_c10|  _c11|  _c12|            _c13|      _c14|  _c15|     _c16|    _c17|           _c18|       _c19|        _c20|            _c21|         _c22|        _c23|     _c24|           _c25|          _c26|        _c27|   _c28|             _c29|
+--------------+-----------+-------------------+-------------------+------------+--------------------+----------+---------------+-------+---------+----+------+------+----------------+----------+------+---------+--------+--------

#5: Base exploration of the data

Basic exploration of the data, some ploting, means etc...

##5.1: Use the PySpark/Spark API